In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import seaborn as sns
from statsmodels import api as sm
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
import numpy as np

In [4]:
df = pd.read_csv( os.path.join('csv_label', 'tsflesh.csv') )
df

,0__variance_larger_than_standard_deviation,0__has_duplicate_max,0__has_duplicate_min,0__has_duplicate,0__sum_values,0__abs_energy,0__mean_abs_change,0__mean_change,0__mean_second_derivative_central,0__median,...,0__query_similarity_count__query_None__threshold_0.0,"0__matrix_profile__feature_""min""__threshold_0.98","0__matrix_profile__feature_""max""__threshold_0.98","0__matrix_profile__feature_""mean""__threshold_0.98","0__matrix_profile__feature_""median""__threshold_0.98","0__matrix_profile__feature_""25""__threshold_0.98","0__matrix_profile__feature_""75""__threshold_0.98",0__mean_n_absolute_max__number_of_maxima_7,original,label
0,0.0,0.0,1.0,1.0,1093.4,59795.32,0.284211,-0.094737,0.022222,54.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,55.600000,55.2,0
1,0.0,0.0,1.0,1.0,1094.7,59935.85,0.278947,-0.100000,-0.013889,54.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,55.600000,54.7,0
2,0.0,0.0,0.0,1.0,1096.3,60107.37,0.294737,-0.084211,-0.019444,54.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,55.600000,54.4,0
3,0.0,0.0,0.0,1.0,1098.0,60289.78,0.278947,-0.068421,-0.008333,54.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,55.600000,54.5,0
4,0.0,0.0,0.0,1.0,1099.1,60408.03,0.268421,-0.036842,-0.016667,54.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,55.600000,54.3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107989,0.0,0.0,0.0,1.0,879.4,38668.30,0.205263,-0.015789,0.013889,43.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,44.228571,44.3,0
107990,0.0,0.0,0.0,1.0,879.7,38694.79,0.205263,-0.015789,-0.011111,43.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,44.257143,44.3,0
107991,0.0,0.0,0.0,1.0,880.0,38721.28,0.184211,0.005263,0.013889,43.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,44.285714,44.3,0
107992,0.0,0.0,0.0,1.0,880.8,38792.96,0.205263,-0.068421,0.027778,43.95,...,0.0,0.89362,2.407434,1.533555,1.194127,1.019286,1.779321,44.400000,45.2,0


In [5]:
features_filter=df[['0__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
       '0__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',
       '0__change_quantiles__f_agg_"var"__isabs_False__qh_0.4__ql_0.0',
       '0__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',
       '0__approximate_entropy__m_2__r_0.5',
       '0__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',
       '0__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',
       '0__cid_ce__normalize_False',
       '0__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',
       '0__approximate_entropy__m_2__r_0.7']]

In [6]:
from tsfresh.feature_extraction.settings import from_columns

settings = from_columns(features_filter)

In [7]:
settings

{'0': {'change_quantiles': [{'f_agg': 'var',
    'isabs': False,
    'qh': 0.8,
    'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.4, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0}],
  'approximate_entropy': [{'m': 2, 'r': 0.5}, {'m': 2, 'r': 0.7}],
  'cid_ce': [{'normalize': False}]}}

In [8]:
df = pd.read_csv( os.path.join('csv_label', 'learning_data_10.csv') )
df=df[df['day']<=2]
df.drop('time', axis=1, inplace=True)
df

,original,label,day
0,53.4,0,0
1,52.8,0,0
2,52.8,0,0
3,53.2,0,0
4,53.6,0,0
...,...,...,...
1151450,53.6,0,2
1151451,53.8,0,2
1151452,53.1,0,2
1151453,53.0,0,2


In [9]:
def make_data(x, str,w):
    x=pd.DataFrame(x)
    window=w
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)
    
#     for i in range(-1, -window, -1):
#         column=f'{str}_lag{i}'
#         x[column]=x[str].shift(i)   
    return x

lag=[] 
window=50
for str in ['original']:
    lag.append(make_data(df[str], str, window))

# lag.append(df['trend'])
# lag.append(df['seasonal'])
# lag.append(df['resid'])
lag.append(df['day'])
lag.append(df['label'])
df_add_lag=pd.concat(lag, axis=1)
df=df_add_lag.dropna().reset_index(drop=True)
df

,original,original_lag1,original_lag2,original_lag3,original_lag4,original_lag5,original_lag6,original_lag7,original_lag8,original_lag9,...,original_lag42,original_lag43,original_lag44,original_lag45,original_lag46,original_lag47,original_lag48,original_lag49,day,label
0,56.0,56.1,56.1,56.2,56.4,55.9,55.7,55.0,54.3,54.0,...,55.7,54.7,54.0,53.6,53.2,52.8,52.8,53.4,0,0
1,55.5,56.0,56.1,56.1,56.2,56.4,55.9,55.7,55.0,54.3,...,55.5,55.7,54.7,54.0,53.6,53.2,52.8,52.8,0,0
2,55.8,55.5,56.0,56.1,56.1,56.2,56.4,55.9,55.7,55.0,...,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,0,0
3,56.2,55.8,55.5,56.0,56.1,56.1,56.2,56.4,55.9,55.7,...,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,0,0
4,56.5,56.2,55.8,55.5,56.0,56.1,56.1,56.2,56.4,55.9,...,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151401,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,56.0,...,49.3,49.2,49.9,50.4,49.2,49.4,48.9,48.2,2,0
1151402,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,...,51.7,49.3,49.2,49.9,50.4,49.2,49.4,48.9,2,0
1151403,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,...,50.6,51.7,49.3,49.2,49.9,50.4,49.2,49.4,2,0
1151404,53.0,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,...,52.4,50.6,51.7,49.3,49.2,49.9,50.4,49.2,2,0


In [10]:
df_val=df[df.columns[:-2]]
df_val

,original,original_lag1,original_lag2,original_lag3,original_lag4,original_lag5,original_lag6,original_lag7,original_lag8,original_lag9,...,original_lag40,original_lag41,original_lag42,original_lag43,original_lag44,original_lag45,original_lag46,original_lag47,original_lag48,original_lag49
0,56.0,56.1,56.1,56.2,56.4,55.9,55.7,55.0,54.3,54.0,...,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,52.8,53.4
1,55.5,56.0,56.1,56.1,56.2,56.4,55.9,55.7,55.0,54.3,...,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8,52.8
2,55.8,55.5,56.0,56.1,56.1,56.2,56.4,55.9,55.7,55.0,...,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2,52.8
3,56.2,55.8,55.5,56.0,56.1,56.1,56.2,56.4,55.9,55.7,...,55.7,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6,53.2
4,56.5,56.2,55.8,55.5,56.0,56.1,56.1,56.2,56.4,55.9,...,55.0,55.7,55.8,55.6,55.7,55.5,55.7,54.7,54.0,53.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151401,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,56.0,...,50.6,51.7,49.3,49.2,49.9,50.4,49.2,49.4,48.9,48.2
1151402,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,56.4,...,52.4,50.6,51.7,49.3,49.2,49.9,50.4,49.2,49.4,48.9
1151403,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,56.8,...,51.0,52.4,50.6,51.7,49.3,49.2,49.9,50.4,49.2,49.4
1151404,53.0,53.1,53.8,53.6,54.3,55.1,55.2,55.6,56.2,56.4,...,51.5,51.0,52.4,50.6,51.7,49.3,49.2,49.9,50.4,49.2


In [11]:
master_df=pd.DataFrame({0:df_val.values.flatten(),
                       1:np.arange(len(df_val)).repeat(df_val.shape[1])})
master_df

,0,1
0,56.0,0
1,56.1,0
2,56.1,0
3,56.2,0
4,56.4,0
...,...,...
57570295,51.7,1151405
57570296,49.3,1151405
57570297,49.2,1151405
57570298,49.9,1151405


In [13]:
from tsfresh.feature_extraction import extract_features
X = extract_features(master_df, column_id=1, default_fc_parameters=settings['0'])

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 50/50 [05:16<00:00,  6.33s/it]


In [18]:
X['original']=df['original']
X['label']=df['label']
X['day']=df['day']

In [19]:
X

,"0__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0","0__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0","0__change_quantiles__f_agg_""var""__isabs_False__qh_0.4__ql_0.0","0__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.0","0__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.4","0__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.0","0__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0",0__approximate_entropy__m_2__r_0.5,0__approximate_entropy__m_2__r_0.7,0__cid_ce__normalize_False,original,label,day
0,0.132243,0.091106,0.087755,0.052986,0.068347,0.118409,0.237931,0.419051,0.362927,2.437212,56.0,0,0
1,0.124600,0.079630,0.071400,0.051803,0.067789,0.115943,0.225000,0.450099,0.337871,2.414539,55.5,0,0
2,0.127666,0.082469,0.074958,0.050429,0.065619,0.117068,0.233333,0.463397,0.348506,2.433105,55.8,0,0
3,0.128116,0.085067,0.072469,0.050429,0.064018,0.117068,0.226667,0.492925,0.347816,2.433105,56.2,0,0
4,0.128430,0.083876,0.071136,0.050079,0.061831,0.115885,0.220690,0.483991,0.345046,2.418677,56.5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151401,0.892681,1.073841,1.171636,0.355094,0.111122,0.727855,0.775862,0.302643,0.179674,6.021628,53.6,0,2
1151402,0.884134,1.064328,1.162222,0.358467,0.111122,0.720816,0.758621,0.336226,0.180011,5.984146,53.8,0,2
1151403,0.897809,1.084590,1.206667,0.358367,0.111122,0.730012,0.765517,0.344838,0.212945,6.004165,53.1,0,2
1151404,0.896479,1.082687,1.202747,0.360258,0.111122,0.729088,0.762069,0.349499,0.249397,6.001666,53.0,0,2


In [20]:
import pandas as pd
import os
import glob

current_dir=os.getcwd()
learning_data_dir=os.path.join(current_dir, 'csv_label')
X.to_csv(os.path.join(learning_data_dir, 'tsflesh_filter.csv'), index=False)